In [1]:
axis_name = {0: 'x', 1: 'y', 2: 'z'}

In [3]:
axis_name[0]

'x'

In [14]:
fig, axes = plt.subplots(1,1)

df = pd.read_csv('../HEWLmd/run1/conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]

# sns.scatterplot(data=df[df['sigma'] <= 0.75e17], x='c', y='sigma', ax=axes)
sns.scatterplot(data=df[df['c'] >= 0.1], x='c', y='sigma', ax=axes)

plt.savefig('HEWL sigma.png')
plt.show()

/n/home13/csun21/miniconda3/envs/md/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # Remove the CWD from sys.path while we load stuff.


In [38]:
elementary_charge

Unit({BaseUnit(base_dim=BaseDimension("charge"), name="elementary charge", symbol="e"): 1.0})

In [37]:
df[df['c']==0.2]

,path,sigma,a,b,c
4,../md_out/ions/0.346M-0.2-a.h5,3.055645e+16,0.346M,1,0.2
10,../md_out/ions/0.172M-0.2-a.h5,-2.214592e+16,0.172M,1,0.2


In [2]:
fig, axes = plt.subplots(3,1, figsize=(12,12))

df = pd.read_csv('../md_out/ions/x_conductivity_results.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]

lesser = df[df['a'] == '0.172M'].copy()

sns.scatterplot(data=lesser[lesser['c'] <= 1.2], x='c', y='sigma', ax=axes[0])
sns.scatterplot(data=lesser[lesser['c'] > 1.2], x='c', y='sigma', ax=axes[1])
sns.scatterplot(data=lesser[lesser['c'] <= 1.2], x='c', y='sigma', ax=axes[2])
axes[0].set_title('0.172M')
axes[0].set(ylabel='S/cm')
axes[0].set(xlabel='')

axes[1].set_title('0.172M')
axes[1].set(ylabel='S/cm')
axes[1].set(xlabel='')

axes[2].set_title('0.346M')
axes[2].set(xlabel='MV/cm')
axes[2].set(ylabel='S/cm')


plt.savefig('ion box along x sigma.png')
plt.show()

/n/home13/csun21/miniconda3/envs/md/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [28]:
fig, axes = plt.subplots(2,1, figsize=(12,20))

df = pd.read_csv('../md_out/ions/results_z.txt', names=['path', 'sigma', 'a', 'b', 'c'])
df['c'] = [float(i[1:-1]) for i in df['c'].tolist()]
df['a'] = [i.split('/')[-1].split('-')[0] for i in df['path'].tolist()]


sns.scatterplot(data=df[df['a'] == '0.346M'], x='c', y='sigma', ax=axes[0])
sns.scatterplot(data=df[df['a'] == '0.172M'], x='c', y='sigma', ax=axes[1])
plt.savefig('ion box along z sigma.png')


In [2]:
import mdtools
from simtk.unit import *
from simtk.openmm.app import ForceField, Topology

import mdtraj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# matplotlib.use('Agg')

from scipy import stats

import re
import sys
import glob
import ast

from new_functions import *

sns.set_theme(style="darkgrid")


In [1]:



atoms = mdtraj.load_frame('../crystalmd/run1/NVT-1.0-a.h5', 0)
lys = atoms.top.select('protein or backbone')
lys_traj = mdtraj.load('../crystalmd/run1/NVT-1.0-a.h5', stride=50)

side = np.mean(lys_traj.unitcell_lengths, axis=0)[2]


x_index = pd.DataFrame(lys_traj.xyz[:,:,0])
y_index = pd.DataFrame(lys_traj.xyz[:,:,1])
z_index = pd.DataFrame(lys_traj.xyz[:,:,2])


diff = x_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    x_index[col] = pd.DataFrame(np.r_[x_index[col].iloc[0], diff[col].iloc[1:]].cumsum())
    
x_index = x_index.dropna()



diff = y_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    y_index[col] = pd.DataFrame(np.r_[y_index[col].iloc[0], diff[col].iloc[1:]].cumsum())
    
y_index = y_index.dropna()

fig, axes = plt.subplots(1,1)

diff = z_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    z_index[col] = pd.DataFrame(np.r_[z_index[col].iloc[0], diff[col].iloc[1:]].cumsum())

z_index = z_index.dropna()

lys_traj.xyz[:,:,0] = x_index.to_numpy()
lys_traj.xyz[:,:,1] = y_index.to_numpy()
lys_traj.xyz[:,:,2] = z_index.to_numpy()





In [13]:
alls = atoms.top.select('all')

In [27]:
lys_traj.superpose(lys_traj, frame=50, atom_indices=, ref_atom_indices= )

In [28]:
aligned.xyz == lys_traj.xyz

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       ...,

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,

In [1]:
import mdtools
from simtk.unit import *
from simtk.openmm.app import ForceField, Topology

import mdtraj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# matplotlib.use('Agg')

from scipy import stats

import re
import sys
import glob
import ast

from new_functions import *

In [2]:
atoms = mdtraj.load_frame('../crystalmd/run1/NVT-1.0-a.h5', 0)
lys = atoms.top.select('protein or backbone')
lys_traj = mdtraj.load('../crystalmd/run1/NVT-1.0-a.h5', atom_indices=lys, stride=50)

In [3]:
lys_traj

<mdtraj.Trajectory with 401 frames, 31360 atoms, 2064 residues, and unitcells at 0x2afdd3d0bd50>

In [4]:
x_index = pd.DataFrame(lys_traj.xyz[:,:,0])
y_index = pd.DataFrame(lys_traj.xyz[:,:,1])
z_index = pd.DataFrame(lys_traj.xyz[:,:,2])

In [6]:
side = np.mean(lys_traj.unitcell_lengths, axis=0)[2]

In [7]:
diff = x_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    x_index[col] = pd.DataFrame(np.r_[x_index[col].iloc[0], diff[col].iloc[1:]].cumsum())
    
x_index = x_index.dropna()



diff = y_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    y_index[col] = pd.DataFrame(np.r_[y_index[col].iloc[0], diff[col].iloc[1:]].cumsum())
    
y_index = y_index.dropna()

fig, axes = plt.subplots(1,1)

diff = z_index.diff()
diff[diff > .95*side] -= side
diff[diff < -.95*side] += side
for col in diff:
    z_index[col] = pd.DataFrame(np.r_[z_index[col].iloc[0], diff[col].iloc[1:]].cumsum())

z_index = z_index.dropna()



In [17]:
lys_traj.xyz[:,:,0] = x_index.to_numpy()
lys_traj.xyz[:,:,1] = y_index.to_numpy()
lys_traj.xyz[:,:,2] = z_index.to_numpy()

In [18]:
lys_traj.save('formatted.pdb')

In [19]:
lys_traj.superpose(lys_traj, ref_atoms_indicies=)

TypeError: superpose() missing 1 required positional argument: 'reference'

In [ ]:
fig, axes = plt.subplots(1,1)
for i in range(10):
sns.lineplot(x=range(len(x_index)), y=x_index[i], ax=axes, palette='mako')
plt.suptitle('x movement')
axes.set(xlabel='frames * 50', ylabel='x coord')
plt.savefig("x.png")

for i in range(10):
sns.lineplot(x=range(len(z_index)), y=z_index[i], ax=axes)
plt.suptitle('z movement')
axes.set(xlabel='frames * 50', ylabel='z coord')
plt.savefig("z.png")

for i in range(10):
    sns.lineplot(x=range(len(y_index)), y=y_index[i], ax=axes, palette='mako')
plt.suptitle('y movement')
axes.set(xlabel='frames * 50', ylabel='y coord')
plt.savefig("y.png")

In [13]:

atoms = mdtraj.load_frame('../crystalmd/run1/NVT-1.0-a.h5', 0)
protein = atoms.top.select('protein')
protein_traj = mdtraj.load('../crystalmd/run1/NVT-1.0-a.h5', atom_indices=protein, stride=50)

In [24]:
rmsds = mdtraj.rmsd(protein_traj, protein_traj, 0) 

In [32]:
fig, axes = plt.subplots(1,1, figsize=(12,8))
sns.scatterplot(x=range(len(rmsds)), y=rmsds, ax=axes)
axes.set(xlabel='frame * 50', ylabel='rmsd')
plt.suptitle('rmsds = mdtraj.rmsd(protein_traj, protein_traj, 0)')
plt.savefig('test.png')

In [6]:
files = glob.glob(f"../crystalmd/irun1/*.h5")
index = []
for i in files:
    index.append(float(re.findall('iter\w+', i)[0][-2:]))

In [2]:
df = pd.read_csv('../crystalmd/new_conduct.txt')

In [45]:
sigmas = []
efs = []

for sigma, ef in zip(df[df.columns[1]].tolist(), df[df.columns[4]].tolist()):
    sigmas.append(float(sigma))
    efs.append(float(ef[1:-1]))

In [46]:
ok = pd.DataFrame()
ok['Conductivity S/cm'] = np.multiply(sigmas, 1e12)
ok['EF MV/cm'] = efs
ok = ok.sort_values('EF MV/cm')

In [47]:
fig, axes = plt.subplots(2,1, figsize=(12,8))
sns.pointplot(data=ok, x='EF MV/cm', y='Conductivity S/cm', ax=axes[0], color='dodgerblue')
sns.scatterplot(data=ok, x='EF MV/cm', y='Conductivity S/cm', ax=axes[1], color='dodgerblue')
axes[0].axhline(534 * 1e6, color='salmon')
axes[1].axhline(534 * 1e6, color='salmon')

plt.suptitle("HEWL EF vs. Conductivity")
plt.savefig("HEWL Ef v Sigma.png")

In [2]:
# morosova to MV/cm
def moroMV(x):
    ef = x * millimeters / (7.53 * nanometers)
    return ef / 1e8

for i in [8.8, 35, 23.4, 68.7]:
    print(moroMV(i))

0.011686586985391765
0.04648074369189906
0.03107569721115537
0.09123505976095617


In [3]:
moroConduct = [157, 53, 534, 461]

In [8]:
atoms = mdtraj.load_frame('../5E21/run1/NVT-1.0-a.h5', 0)
ions = atoms.top.select('!protein and !water')
ion_index = mdtraj.load('../5E21/run1/NVT-1.0-a.h5', atom_indices=ions, stride=500)

In [9]:
# lysozyme volume
vol = np.mean(ion_index.unitcell_volumes)
vol = vol * 1e-24
vol

7.003082275390624e-22

In [13]:
na = 0
cl = 0

for i in ion_index.top.atoms:
#     print(str(i)[-2:])
    if str(i)[-2:] == 'NA':
        na += 1
    else:
        cl += 1
        
# cl /= 6.022e23
# na /= 6.022e23

# cl /= vol
# na /= vol

# cl *= 1e3
# na *= 1e3

In [15]:
na

0

In [12]:
print('Na', na, 'mM')
print('Cl', cl, 'mM')

Na 0.0 mM
Cl 75.87872040575155 mM


In [ ]:
atoms.unitcell_lengths

In [ ]:
atom = pd.DataFrame(atoms.top.atoms)

In [ ]:

nowater = mdtraj.load_frame('../crystalmd/NVT_run-2.h5', 0, atom_indices=yui)

In [ ]:
atomnowater = pd.DataFrame(nowater.top.atoms)

In [ ]:
atomnowater[0].tolist()

In [ ]:
atomnowater = pd.DataFrame(nowater.top.atoms)
nowater.top.select('!water and not name CA and not name LYS1 and not name C and not name O and not name CB')

In [ ]:
ok = mdtraj.load('../crystalmd/NVT_run-2.h5', 0, atom_indices=cl, stride=100)

In [ ]:
cl = pd.DataFrame(ok.top.select("name Cl"))[0].tolist()

In [ ]:
ok

In [ ]:
huh = pd.read_csv('../crystalmd/NVT_run-2.csv')

In [ ]:
huh

In [ ]:
sns.lineplot(data=huh, x='Time (ps)', y='Temperature (K)')

In [ ]:
import mdtools
from simtk.unit import *
from simtk.openmm.app import ForceField, Topology
import mdtraj
import numpy as np
from scipy.constants import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import StrMethodFormatter

import glob
# from functions import *
from pathlib import Path

import matplotlib
# matplotlib.use('Agg')

from scipy import stats
import seaborn as sns
import re
import sys

In [ ]:
import pandas as pd
import glob
import re
import os
import numpy as np


In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
glob.glob('../txt_out/efield*.txt')

In [ ]:
df = pd.concat(map(pd.read_csv, glob.glob('../txt_out/efield*.txt')))
df

In [ ]:
suffix = 'c'

h5s = glob.glob(f"../md_out/efield/*-{suffix}.h5")
a = [float(i[0][:-1]) for i in [re.findall("[-0-9.]+-", i) for i in h5s]]

pathway = [(i, j) for i,j in zip(h5s, a)]
pathway.sort(key=lambda x: x[1])
pathway

In [ ]:
suffix = 'a'

h5s = glob.glob(f"../crystalmd/run_{suffix}/*.h5")
print(h5s)
a = [i for i in [re.findall("[0-9.]+-", i) for i in h5s]]
print(a)
pathway = [(i, j) for i,j in zip(h5s, a)]
pathway.sort(key=lambda x: x[1])
pathway

In [ ]:
glob.glob('../txt_out/*.txt')

In [ ]:
pd.read_csv('../txt_out/efield-04.txt')

In [ ]:
ef[0]._value

In [ ]:
from simtk.unit import *
ef = 5*(kilojoule_per_mole/nanometer/elementary_charge)
ef = (ef.in_units_of(volts/centimeter/mole)/AVOGADRO_CONSTANT_NA)
print(ef)                           # prints Quantity(value=518213.4402345789, unit=volt/centimeter)


In [ ]:
df['efield'] = df['efield']*(kilojoule_per_mole/nanometer/elementary_charge)
df['efield'] = df['efield'].in_units_of(volts/centimeter/mole)/AVOGADRO_CONSTANT_NA
df


In [ ]:
sns.histplot(pd.read_csv('NPT_run_a.csv')['Box Volume (nm^3)'])
plt.axvline(474.3)

In [ ]:
df = pd.concat((pd.read_csv(f, names=['path', 'conductivity', 'efield']) for f in glob.glob('../txt_out/[efield]*.txt')), join='outer')
df.loc[df['efield'] == 2]

In [ ]:
df = pd.concat((pd.read_csv(f, names=['path', 'conductivity', 'efield']) for f in glob.glob('../txt_out/[efield]*.txt')), join='outer')
df = df.drop(df[df['efield'] == 0].index)

ok = []
for i in df[['path']].to_numpy():
    ok.append(int(i[0].split('/')[-1][:-5]))
    
df[['efield']] = ok

df = df.drop(columns=['path'])

df = df.astype('float')

df = df.sort_values('efield')


for i in range(27):
    df.iloc[i]['conductivity'] = np.abs(df.iloc[i]['conductivity'])

ef = df[['efield']].to_numpy(dtype=object).flatten()
for i in range(len(ef)):
    
    ef[i] = ef[i]*(kilojoule_per_mole/nanometer/elementary_charge)
    ef[i] = ef[i].in_units_of(volts/centimeter/mole)/AVOGADRO_CONSTANT_NA
    ef[i] /= 10e6
    ef[i] = np.round(ef[i]._value, 2)
df[['efield']] = ef


display(df)
fig, axs = plt.subplots(1, 1, figsize=(10,5), dpi=100)
sns.pointplot(x='efield', y='conductivity', data=df, join=False, dodge=True, ci='sd', ax=axs)
axs.set(xlabel="Efield (MV/cm)", ylabel = "Conductivity (mS/cm)")
axs.axhline(0)
plt.show()

df[['conductivity']] = df[['conductivity']].abs()

fig, axs = plt.subplots(1, 1, figsize=(10,5), dpi=100)
sns.pointplot(x='efield', y='conductivity', data=df, join=False, dodge=True, ci='sd', ax=axs)
axs.set(xlabel="Efield (MV/cm)", ylabel = "Conductivity (mS/cm)")
plt.show()

In [ ]:
ok = pd.read_csv('run_b.csv')
plt.figure(dpi=150, figsize=(8,3))
sns.lineplot(data=ok, x='Time (ps)', y='Total Energy (kJ/mole)', palette='Set1', color='dodgerblue')
plt.title('Production Energy Change (run_a)')
plt.show()

In [ ]:
ok = pd.read_csv('run_b.csv')
plt.figure(dpi=150, figsize=(8,3))
sns.lineplot(data=ok, x='Time (ps)', y='Box Volume (nm^3)', palette='Set1', color='salmon')
plt.title('Production Volume Change (run_b)')
plt.show()